In [14]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

import numpy as np
from tqdm.auto import tqdm

In [2]:
State = namedtuple('State', ['taken', 'not_taken'])


def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

In [3]:
PROBLEM_SIZE = 8
NUM_SETS = 16
SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [4]:
SETS

(array([False,  True, False, False, False, False, False, False]),
 array([False, False,  True,  True,  True, False, False,  True]),
 array([ True, False, False, False,  True,  True, False, False]),
 array([ True, False, False, False, False, False, False,  True]),
 array([False,  True, False,  True, False, False, False,  True]),
 array([False, False, False, False, False, False,  True, False]),
 array([False, False, False,  True, False, False, False, False]),
 array([False, False,  True, False, False, False, False, False]),
 array([False, False,  True, False, False, False, False, False]),
 array([False, False,  True,  True, False, False, False,  True]),
 array([False, False, False,  True, False, False, False,  True]),
 array([False, False, False, False, False, False, False, False]),
 array([False, False, False, False, False, False, False, False]),
 array([ True, False, False, False, False,  True, False, False]),
 array([False, False, False, False,  True,  True, False, False]),
 array([Fa

In [11]:
def h(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


# This is a greedy approach, not as efficient as h3, but often returns results with the same number of tiles 
# with fewer steps, although sometimes it returns results with more tiles depending on the set."
def h_greedy(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0  # If all the elements are already cover the remaining cost is 0
    missing_size = PROBLEM_SIZE - np.sum(already_covered)

    taken = 0
    while np.any(already_covered == False) and taken < missing_size:
        #find the set with the maximum cover left
        max_covering_set = None
        max_covering_size = -1
        for i, s in enumerate(SETS):
            if i not in state.taken:
                covering_size = np.sum(np.logical_and(s, np.logical_not(already_covered)))
                if covering_size > max_covering_size:
                    max_covering_size = covering_size
                    max_covering_set = s
        
        #then adds the founded set
        if max_covering_set is not None:
            state.taken.add(i)
            already_covered = np.logical_or(already_covered, max_covering_set)
            taken += 1
        else:
            break
    
    return taken



def f(state):
    return len(state.taken) + h3(state)



In [12]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
           
        _, current_state = frontier.get()
        print(current_state)
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

48it [00:00, 1682.54it/s]

State(taken={0}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15})
State(taken={2}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15})
State(taken={13}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15})
State(taken={14}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15})
State(taken={5}, not_taken={0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15})
State(taken={1}, not_taken={0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15})
State(taken={4}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15})
State(taken={9}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15})
State(taken={15}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14})
State(taken={7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15})
State(taken={0, 15}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14})
State(taken={1, 2}, not_taken={0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15})
State(taken={13, 15}, not_taken={0, 1, 2

In [13]:
current_state

State(taken={0, 9, 2, 15}, not_taken={1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14})